In [1]:
import pandas as pd
from ast import literal_eval
from unidecode import unidecode
from collections import OrderedDict
import argparse
from difflib import SequenceMatcher
from sklearn.model_selection import train_test_split
import os
import itertools

In [2]:
def returnContext(prev, target, next, prev_nr, next_nr):
    if (prev_nr == 0):
        prev = []
    elif (prev_nr < len(prev)):
        prev = prev[-prev_nr:]
    if (next_nr == 0):
        next = []
    elif (next_nr < len(next)):
        next = next[:next_nr]
    return list(itertools.chain(prev, target, next))

In [4]:

    
DATA_DIR = '../data'
DATA_SET = 'multicite'
OUTPUT_CAT = ['t','bt','tn','btn','bbt','tnn']

DATASET_DIR = DATA_DIR + f'/{DATA_SET}'

for cat in OUTPUT_CAT:
    if not os.path.exists(os.path.join(DATASET_DIR, f'{DATA_SET}_fc_{cat}')):
        os.makedirs(os.path.join(DATASET_DIR, f'{DATA_SET}_fc_{cat}'))

for dataset in ["train", "test"]:
    data_df = pd.read_csv(DATASET_DIR + f"/{dataset}_raw.txt", sep="\t", engine="python", dtype=object)
    data_df['cite_context_paragraph'] = data_df['cite_context_paragraph'].apply(eval)
    result_t = pd.DataFrame(columns=['CC','label'])
    result_bt = pd.DataFrame(columns=['CC','label'])
    result_tn = pd.DataFrame(columns=['CC','label'])
    result_btn = pd.DataFrame(columns=['CC','label'])
    result_bbt = pd.DataFrame(columns=['CC','label'])
    result_tnn = pd.DataFrame(columns=['CC','label'])
    for idx, row in data_df.iterrows():
        paragraph = row['cite_context_paragraph']
        citation_context = [row['citation_context']]
        
        try:
            citing_sent_index = paragraph.index(citation_context[0])
            context_prev = paragraph[:citing_sent_index]
            context_next =  paragraph[citing_sent_index +1:]      
        except (IndexError, ValueError) as e:
            print(idx)
            print(citation_context)
            print('Index not found!')
        result_t.loc[len(result_t)] = [returnContext(context_prev, citation_context, context_next, 0, 0), row['citation_class_label']]
        result_bt.loc[len(result_bt)] = [returnContext(context_prev, citation_context, context_next, 1, 0), row['citation_class_label']]
        result_tn.loc[len(result_tn)] = [returnContext(context_prev, citation_context, context_next, 0, 1), row['citation_class_label']]
        result_btn.loc[len(result_btn)] = [returnContext(context_prev, citation_context, context_next, 1, 1), row['citation_class_label']]
        result_bbt.loc[len(result_bbt)] = [returnContext(context_prev, citation_context, context_next, 2, 0), row['citation_class_label']]
        result_tnn.loc[len(result_tnn)] = [returnContext(context_prev, citation_context, context_next, 0, 2), row['citation_class_label']]
    result_t.to_csv(f'{DATASET_DIR}/{DATA_SET}_fc_t/{dataset}.csv', index=False)
    result_bt.to_csv(f'{DATASET_DIR}/{DATA_SET}_fc_bt/{dataset}.csv', index=False)
    result_tn.to_csv(f'{DATASET_DIR}/{DATA_SET}_fc_tn/{dataset}.csv', index=False)
    result_btn.to_csv(f'{DATASET_DIR}/{DATA_SET}_fc_btn/{dataset}.csv', index=False)
    result_bbt.to_csv(f'{DATASET_DIR}/{DATA_SET}_fc_bbt/{dataset}.csv', index=False)
    result_tnn.to_csv(f'{DATASET_DIR}/{DATA_SET}_fc_tnn/{dataset}.csv', index=False)